In [1]:
import itertools
import pandas as pd
import numpy as np

## Enter Filename, Date and Title:

In [2]:
data = pd.read_csv("scoresheets/20-06-17 Chaos.csv", sep="\t", comment="#")
date = "20-06-17"
# title is used to only run code pertaining to given disc golf event:
title = "Chaos"

complete_matchups = pd.read_csv("Completed Matchups/20-07-18.csv", sep="\t", names=["timestamp", "p1", "p2", "winner", "point diff", "hole num"])

non_participants = ["Jamie"]

display(data)

,Amy,Ben,Ellie,Levi,Louis,Matthew,Max,Mo,Will
0,4.0,NaN,7,7,3,NaN,8,4,4
1,20.0,14.0,18,18,16,NaN,14,10,20
2,12.0,9.0,14,8,9,10.0,8,6,9
3,NaN,17.0,32,22,21,20.0,19,15,23


In [3]:
# get all names and name pairs:
names = list(data.columns.values)
match_pairs = tuple(itertools.combinations(names, 2))

In [4]:
# Calcs total shots for each player for current scoresheet:
def get_totals(data):
    score_totals = []
    # Calc sum for each column:
    [ score_totals.append(data[col].sum()) for col in data.columns ]

    # Create and populate dataframe for totals:
    score_totals_df = pd.DataFrame(columns=[data.columns])
    score_totals_df.loc[0] = score_totals
    
    print("Score totals:")
    return score_totals_df

In [5]:
get_totals(data)

Score totals:


,Amy,Ben,Ellie,Levi,Louis,Matthew,Max,Mo,Will
0,36.0,40.0,71.0,55.0,49.0,30.0,49.0,35.0,56.0


In [6]:
def get_scores(data):
    scores = pd.DataFrame(columns=["p1", "p2", "p1 score", "p2 score", "rounds played"])
    #scores = dict()
    
    # loop through name combos:
        # get indices of NaN for both people
        # loop through rounds:
            # if either player has NaN on current round, skip it
            # else increment rounds played, add shots of round to sums
    
    for (p1, p2) in match_pairs:
        p1_score = 0
        p2_score = 0
        rounds_played = 0
        
        #print(f"{p1}, {p2}")
        
        p1_na_rounds = list(data[pd.isna(data[p1])].index.values)
        p2_na_rounds = list(data[pd.isna(data[p2])].index.values)
        
        #print(f"{p1_na_rounds}, {p2_na_rounds}")
        
        #print(f"NaN rounds: {p1_na_rounds + p2_na_rounds}")
        
        for r in range(data.shape[0]):
            if r in p1_na_rounds + p2_na_rounds:
                continue
            
            p1_score += data[p1][r]
            p2_score += data[p2][r]
            rounds_played += 1
        
        scores.loc[len(scores)] = [p1, p2, p1_score-p2_score, p2_score-p1_score, rounds_played]
        
        #print(f"{p1} vs {p2}, {p1_score} to {p2_score}; {rounds_played} rounds played")
    return scores

In [7]:
# Returns the name of the loser of a match, given a Series of a scoresheet:
def loser(row):
    if row["winner"] == row["p1"]:
        return row["p2"]
    elif row["winner"] == row["p2"]:
        return row["p1"]
    else:
        return "Draw"

In [8]:
# Convert scores into format for form results spreadsheet:
def get_form_inputs(scores):
    # p1, p2, winner, point diff, hole num
    form_inputs = pd.DataFrame(columns=["timestamp", "p1", "p2", "winner", "point diff", "hole num"])
    
    for i, row in scores.iterrows():
        if (row["p1 score"] < row["p2 score"]):
            winner = row["p1"]
        else:
            winner = row["p2"]
        form_inputs.loc[len(form_inputs)] = [date, row["p1"], row["p2"], winner, abs(row["p1 score"]), row["rounds played"]]
    return form_inputs

## Get scores and form inputs:

In [9]:
scores = get_scores(data)
# display(scores)

In [10]:
# Get scores in results spreadsheet format:
f_scores = get_form_inputs(scores)
f_scores

,timestamp,p1,p2,winner,point diff,hole num
0,20-06-17,Amy,Ben,Ben,9.0,2
1,20-06-17,Amy,Ellie,Amy,3.0,3
2,20-06-17,Amy,Levi,Levi,3.0,3
3,20-06-17,Amy,Louis,Louis,8.0,3
4,20-06-17,Amy,Matthew,Matthew,2.0,1
5,20-06-17,Amy,Max,Max,6.0,3
6,20-06-17,Amy,Mo,Mo,16.0,3
7,20-06-17,Amy,Will,Will,3.0,3
8,20-06-17,Ben,Ellie,Ben,24.0,3
9,20-06-17,Ben,Levi,Ben,8.0,3


In [11]:
# Get form inputs concerning Ben, since he has left for Leeds:
if (title=="Chaos"):
    ben_matches = f_scores[ (f_scores["p1"]=="Ben") | (f_scores["p2"]=="Ben")]
    display(ben_matches)

,timestamp,p1,p2,winner,point diff,hole num
0,20-06-17,Amy,Ben,Ben,9.0,2
8,20-06-17,Ben,Ellie,Ben,24.0,3
9,20-06-17,Ben,Levi,Ben,8.0,3
10,20-06-17,Ben,Louis,Ben,6.0,3
11,20-06-17,Ben,Matthew,Ben,4.0,2
12,20-06-17,Ben,Max,Ben,1.0,3
13,20-06-17,Ben,Mo,Mo,9.0,3
14,20-06-17,Ben,Will,Ben,12.0,3


In [12]:
# Eliminate any matches which have already taken place:
# NOTE: NO LONGER USED, since repeated matchups now contribute to previous score
def remove_completed_mus(scores):
    filtered_inputs = scores
    
    # iterate through scoresheet:
    for i, row in scores.iterrows():
        # Get DataFrames for both situations where current scoresheet matchup could appear in completed matchups:
        comp_a = complete_mus[(complete_mus["p1"] == row["p1"]) & (complete_mus["p2"] == row["p2"])]
        comp_b = complete_mus[(complete_mus["p2"] == row["p1"]) & (complete_mus["p1"] == row["p2"])]
        
        #display(comp_a)
        #display(comp_b)
        
        # if either DataFrame is not empty, filter out matchup from scoresheet:
        if not comp_a.empty:
            #print("comp a")
            # NOT (p1=p1 AND p2=p2)  <- Get rid of duplicate matchup
            # Equivalent to (De Morgan's Law!):
            # p1!=p1 OR p2!=p2
            filtered_inputs = filtered_inputs[(filtered_inputs["p1"] != row["p1"]) | (filtered_inputs["p2"] != row["p2"])]
        elif not comp_b.empty:
            #print("comp b")
            filtered_inputs = filtered_inputs[(filtered_inputs["p2"] != row["p1"]) | (filtered_inputs["p1"] != row["p2"])]
        #else:
            #print("Not duplicate")
        
    return filtered_inputs

In [13]:
# filtered_inputs = remove_completed_mus(f_scores)
# filtered_inputs

In [14]:
def update_rematches(scores, complete_matchups):
    # loop through scoresheet:
        # if repeat matchup found:
            # calc new point diff (in correct direction)
            # if new point diff is negative:
                # change winner
            # else if new point diff is 0:
                # change to draw
            # increment hole num correctly
            # change timestamp
    
    # create deep copy of complete matchups, to avoid issues with running cells more than once
    complete_mus = complete_matchups.copy()
    
    # loop through new scoresheet:
    for i, row in scores.iterrows():
        # get DataFrames for repeat of current matchup, from completed matchups:
        poss_a = complete_mus[(complete_mus["p1"] == row["p1"]) & (complete_mus["p2"] == row["p2"])]
        poss_b = complete_mus[(complete_mus["p2"] == row["p1"]) & (complete_mus["p1"] == row["p2"])]
        
        dup_mu = None
        
        print("------------\n", row["p1"], ", ", row["p2"])
        
        # put duplicate matchup in own dataframe, if it exists:
        if not poss_a.empty:
            print("poss a")
            dup_mu = poss_a.iloc[0]
        elif not poss_b.empty:
            print("poss b")
            dup_mu = poss_b.iloc[0]
        else:
            print("- No duplicate MU \n")
            # Add matchup to completed matchups:
            complete_mus.loc[len(complete_mus)] = row
            continue
        
        # ----- The following only happens if duplicate matchup found: -----
        
        # if winner of prev match is same as winner of new match, increment point diff
        # else decrement point diff
            # if new point diff is negative, change winner
            # if new point diff is 0, change winner to Draw
            
        new_point_diff = None
        ind = dup_mu.name    # index of current duplicate matchup in completed matchups
        
        print("Previous match:")
        display(dup_mu.to_frame().transpose())
        
        print("New match:")
        display(row.to_frame().T)
        
        
        # if winner is same in both matches, increment point diff:
        if dup_mu["winner"] == row["winner"]:
            new_point_diff = complete_mus.loc[ind]["point diff"] + row["point diff"]
            print(f"Same winner ({dup_mu['winner']}), point diff={new_point_diff}")
            
        # if winner was different, decrement point diff:
        else:
            new_point_diff = complete_mus.loc[ind]["point diff"] - row["point diff"]
            print(f"Different match winner ({row['winner']} as opposed to {dup_mu['winner']}), point diff={new_point_diff}")
            
            # point diff now meaning a draw:
            if new_point_diff == 0:
                complete_mus.loc[ind, "winner"] = "Draw"
                print("It is now a draw!")
                
            # point diff now meaning a different winner:
            elif new_point_diff < 0:
                complete_mus.loc[ind, "winner"] = row["winner"]
                new_point_diff = -new_point_diff
                print(f"New matchup winner: {row['winner']}! Point diff={new_point_diff}")
            
        # update point diff, hole num and timestamp:
        complete_mus.loc[ind, "point diff"] = new_point_diff
        complete_mus.loc[ind, "hole num"] += row["hole num"]
        complete_mus.loc[ind, "timestamp"] = date
        
        # --- NOTE: HOW TO AVOID SettingWithCopyWarning ---
        # BAD:  complete_mus.loc[ind]["point diff"] += new_point_diff
        # GOOD: complete_mus.loc[ind, "point diff"] += new_point_diff
        
        print("\n")
    
    return complete_mus

In [15]:
final_standings = update_rematches(f_scores, complete_matchups)

------------
 Amy ,  Ben
poss a
Previous match:


,timestamp,p1,p2,winner,point diff,hole num
1,17/06/20,Amy,Ben,Ben,9,2


New match:


,timestamp,p1,p2,winner,point diff,hole num
0,20-06-17,Amy,Ben,Ben,9,2


Same winner (Ben), point diff=18.0


------------
 Amy ,  Ellie
poss a
Previous match:


...

Same winner (Max), point diff=4.0


------------
 Matthew ,  Mo
poss a
Previous match:


,timestamp,p1,p2,winner,point diff,hole num
23,16/07/20,Matthew,Mo,Mo,6,6


New match:


,timestamp,p1,p2,winner,point diff,hole num
31,20-06-17,Matthew,Mo,Mo,9,2


Same winner (Mo), point diff=15.0


------------
 Matthew ,  Will
- No duplicate MU 

------------
 Max ,  Mo
- No duplicate MU 

------------
 Max ,  Will
- No duplicate MU 

------------
 Mo ,  Will
- No duplicate MU 



In [16]:
final_standings

,timestamp,p1,p2,winner,point diff,hole num
0,20-06-17,Louis,Levi,Draw,0.0,13.0
1,20-06-17,Amy,Ben,Ben,18.0,4.0
2,20-06-17,Ben,Ellie,Ben,48.0,6.0
3,20-06-17,Ben,Levi,Ben,16.0,6.0
4,20-06-17,Ben,Louis,Ben,12.0,6.0
5,20-06-17,Ben,Max,Ben,2.0,6.0
6,20-06-17,Ben,Will,Ben,24.0,6.0
7,20-06-17,Ben,Mo,Mo,11.0,NaN
8,20-06-17,Matthew,Ben,Ben,11.0,NaN
9,20-06-17,Louis,Amy,Louis,5.0,8.0


In [17]:
test_scores = pd.read_csv("test_scores.csv", sep="\t", names=["timestamp", "p1", "p2", "winner", "point diff", "hole num"])
# display(test_scores
test_complete_mus = pd.read_csv("test_comp_mus.csv", sep="\t", names=["timestamp", "p1", "p2", "winner", "point diff", "hole num"])

# update_rematches(test_scores, test_complete_mus)

In [18]:
# Filter out non-participants, as defined at top of notebook:

f_final_standings = final_standings.copy()

for name in non_participants:
    f_final_standings = f_final_standings[(f_final_standings["p1"] != name) & (f_final_standings["p2"] != name)]
f_final_standings

,timestamp,p1,p2,winner,point diff,hole num
0,20-06-17,Louis,Levi,Draw,0.0,13.0
1,20-06-17,Amy,Ben,Ben,18.0,4.0
2,20-06-17,Ben,Ellie,Ben,48.0,6.0
3,20-06-17,Ben,Levi,Ben,16.0,6.0
4,20-06-17,Ben,Louis,Ben,12.0,6.0
5,20-06-17,Ben,Max,Ben,2.0,6.0
6,20-06-17,Ben,Will,Ben,24.0,6.0
7,20-06-17,Ben,Mo,Mo,11.0,NaN
8,20-06-17,Matthew,Ben,Ben,11.0,NaN
9,20-06-17,Louis,Amy,Louis,5.0,8.0


In [19]:
my_df = pd.DataFrame(columns=["Name", "Age"])
my_df.loc[len(my_df)] = ["Matthew", 22]
my_df.loc[len(my_df)] = ["Jeremy", 25]
my_df.loc[len(my_df)] = ["Laura", 21]
my_df.loc[0]["Age"] +=2
my_df

,Name,Age
0,Matthew,24
1,Jeremy,25
2,Laura,21
